# Welcome to the OGD.AT Lab!

In [ ]:
from os.path import exists
from urllib.request import urlretrieve
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import hvplot.pandas

## Fetching geodata from data.wien.gv.at WFS

In [ ]:
SIZE = 400

In [ ]:
def gdf_from_wfs(layer):
    file = f'{layer}.json'
    url = f"https://data.wien.gv.at/daten/geo?service=WFS&request=GetFeature&version=1.1.0&typeName=ogdwien:{layer}&srsName=EPSG:4326&outputFormat=json"
    if not exists(file):
        urlretrieve(url, file)
    return gpd.read_file(file)

In [ ]:
gdf = gdf_from_wfs('ELADESTELLEOGD')

In [ ]:
gdf.hvplot(geo=True, tiles='OSM', width=SIZE, height=SIZE, hover_cols=['DESIGNATION']).opts(active_tools=['wheel_zoom'])

In [ ]:
def hvplot_with_buffer(gdf, buffer_size, title=''):
    buffered = gdf.to_crs('epsg:31287').buffer(buffer_size)
    buffered = gdf.copy().set_geometry(buffered).to_crs('epsg:4326')
    
    plot = ( buffered.hvplot(geo=True, title=title, tiles='OSM', width=SIZE, height=SIZE, alpha=0.5, line_width=0) * 
      gdf.hvplot(geo=True, hover_cols=['DESIGNATION']) 
    ).opts(active_tools=['wheel_zoom'])
    
    return plot

In [ ]:
hvplot_with_buffer(gdf, 100)

In [ ]:
gdf2 = gdf_from_wfs('CITYBIKEOGD')

In [ ]:
hvplot_with_buffer(gdf, 100, title='EV Charging Stations') + hvplot_with_buffer(gdf2, 100, title='Citybike Stations')

## Visualize features around an address

In [ ]:
import geopy
from geopy.geocoders import Nominatim

In [ ]:
address = "Giefinggasse 2, 1210 Wien"

In [ ]:
locator = Nominatim(user_agent="myGeocoder")
location = locator.geocode(address)

In [ ]:
print(location.address)
print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))

In [ ]:
geocoded_gdf = gpd.GeoDataFrame(pd.DataFrame([
    {'geometry': Point(location.longitude, location.latitude), 'address': address}
])).set_crs('epsg:4326')

In [ ]:
hvplot_with_buffer(geocoded_gdf, 1000, title='EV Charging Stations') * gdf.hvplot(geo=True).opts(active_tools=['wheel_zoom']) 